In [1]:
import datetime

import pandas as pd
import geopandas as gpd

import plotly.express as px

px.set_mapbox_access_token("pk.eyJ1IjoiamdhY29zdGFzIiwiYSI6ImNsYWJwd3g1ZDAwaGUzb3Q0ZG04NDNndGgifQ.brk6kVA6biVSH0ovZ1dreA")

In [3]:
df_data = pd.read_csv('../data/comed_month/comed_202001.csv')
df_data['date_time'] = pd.to_datetime(df_data.date_time)
df_data = df_data[df_data.service_name.str.contains('RESIDENTIAL')]
df_data = df_data.groupby(['zip5', 'date_time']).energy.sum().reset_index()

day = 31
df_data = df_data[(df_data.date_time.dt.day==day)].copy()

gdf_zc = gpd.read_file('../data/geo/Chicago_ZC.geojson')
gdf_zc['GEOID20'] = gdf_zc['GEOID20'].astype(int)

gdf_data = pd.merge(gdf_zc, df_data, left_on='GEOID20', right_on='zip5')


In [4]:
hour = 19
minute = 0

#gdf_plot = gdf_data[(gdf_data.date_time.dt.hour==hour) & (gdf_data.date_time.dt.minute==minute)].copy().set_index('GEOID20')
gdf_plot = gdf_data.copy().sort_values('date_time')#.set_index('GEOID20')
#gdf_plot = gdf_data[(gdf_data.date_time.dt.hour>=12) & (gdf_data.date_time.dt.hour<=20)].copy().sort_values('date_time')

gdf_plot['time'] = gdf_plot['date_time'].dt.time

df_max_en = gdf_plot.groupby('zip5').energy.max().reset_index().rename(columns={'energy': 'max_energy'})

gdf_plot = pd.merge(gdf_plot, df_max_en, on='zip5')

gdf_plot['energy_norm'] = gdf_plot.energy / gdf_plot.max_energy

In [5]:
fig = px.choropleth_mapbox(gdf_plot,
                   geojson=gdf_zc,
                   featureidkey='properties.GEOID20',
                   color="energy_norm",
                   locations='GEOID20',
                   width=1000,
                   height=700,
                   center={'lat':41.6, 'lon':-88.99},
                   zoom=6,
                   range_color=(0, 1),
                   mapbox_style='carto-positron',
                   animation_frame='time',
                   opacity=0.65,
                   )
fig.update_geos(fitbounds="locations", visible=False)
#fig.show()
fig.write_html('20180131_animation.html')

In [ ]:
gdf_places = gpd.read_file('data/census_places/tl_2019_17_place.shp')

In [ ]:
gdf_zc.shape

In [ ]:
gdf_places.head()

In [ ]:
fig = px.choropleth_mapbox(gdf_places,
                   geojson=gdf_places,
                   featureidkey='properties.GEOID',
                   #color="energy_norm",
                   locations='GEOID',
                   width=1000,
                   height=700,
                   center={'lat':41.6, 'lon':-88.99},
                   zoom=6,
                   range_color=(0, 1),
                   mapbox_style='carto-positron',
                   opacity=0.65,
                   )
fig.update_geos(fitbounds="locations", visible=False)
#fig.show()